In [1]:
device="mps"


In [2]:
from typing import List
import torch
import tqdm.notebook as tqdm

In [3]:
with open("data/tinystories/the_complete_works_of_william_shakespeare.txt") as f:
    corpus = f.read()

In [4]:
split = 0.8
train_text = corpus[:int(len(corpus)*split)]
test_text = corpus[int(len(corpus)*split):]

print(f"Training Size: {len(train_text):,} chars")
print(f"Test Size: {len(test_text):,} chars")

Training Size: 4,302,929 chars
Test Size: 1,075,733 chars


In [5]:
def encode(list: List[str]) -> torch.Tensor:
    result = []
    for string in list:
        result.append([ord(c) for c in string])
    return torch.tensor(result)

def decode(arr: torch.Tensor) -> str:
    arr = arr.tolist()
    result = []
    for string in arr:
        result.append("".join([chr(c) for c in string]))
    return result

def get_batch(batch_size, seq_length, use_train=True):
    if use_train:
        data = train_text
    else:
        data = test_text
    
    start_idx = torch.randint(0, len(data) - seq_length, (batch_size,))

    batch = []
    for i in start_idx:
        batch.append(encode([data[i:i+seq_length]]))
        
    return torch.stack(batch).view(batch_size, seq_length)
    

print(encode(["hello", "world"]))
print(decode(encode(["hello", "world"])))
print(get_batch(2, 5, use_train=False))

tensor([[104, 101, 108, 108, 111],
        [119, 111, 114, 108, 100]])
['hello', 'world']
tensor([[105, 115,  32, 110, 101],
        [114, 116, 115,  32, 111]])


In [6]:
import torch
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # LSTM layer
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        
        # Fully connected layer to predict each character
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden=None):
        # Embedding
        x = self.embedding(x)
        
        # Initialize hidden state and cell state if not provided
        if hidden is None:
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim, device=x.device)
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim, device=x.device)
            hidden = (h0, c0)
        
        # LSTM output along with new hidden state
        out, hidden = self.lstm(x, hidden)
        
        # Reshape output for the fully connected layer
        out = out.reshape(-1, self.hidden_dim)
        out = self.fc(out)
        return out, hidden

# Create an instance of the updated model
vocab_size = 256  # number of unique characters
embed_dim = 64    # embedding dimension
hidden_dim = 128  # LSTM hidden dimensions
num_layers = 4    # number of LSTM layers

model = Model(vocab_size, embed_dim, hidden_dim, num_layers).to(device)
print(f"Model uses {sum(p.numel() for p in model.parameters()):,} parameters")
print(f"Model uses {sum(p.numel() for p in model.parameters() if p.requires_grad):,} trainable parameters")


Model uses 545,024 parameters
Model uses 545,024 trainable parameters


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.9)


In [8]:
# Validation
@torch.no_grad()
def evaluate():
    model.eval()
    batch = get_batch(128, 256, use_train=False).to(device)
    x = batch[:,:-1]
    y = batch[:,1:]

    output, _ = model(x)
    loss = criterion(output, y.flatten())
    return loss.item()

evaluate()

5.5366716384887695

In [9]:
pbar = tqdm.tnrange(10_000)
for step in pbar:
    model.train()
    # Encode the sequence
    batch = get_batch(64, 128).to(device)
    x = batch[:,:-1]
    y = batch[:,1:]
        
    # Forward pass
    output, _ = model(x)
    loss = criterion(output, y.flatten())
        
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # scheduler.step()

    if step % 100 == 0:
        val_loss = evaluate()
        
    pbar.set_postfix_str(f"Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}")

  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import torch.nn.functional as F

def temperature_sampling(logits, temperature=0.8):
    # Scale logits by temperature
    scaled_logits = logits / temperature
    # Convert logits to probabilities
    probs = F.softmax(scaled_logits, dim=-1)
    # Sample from the probabilities
    return torch.multinomial(probs, num_samples=1)

# Validation Loop with Temperature Sampling
model.eval()
initial_input = encode(['First Citizen:']).to(device)
generated_text = []
hidden = None  # Hidden state initialization

with torch.no_grad():
    for _ in range(512):  # Generate 512 characters
        output, hidden = model(initial_input, hidden)  # Ensure model accepts and returns hidden state
        predicted = temperature_sampling(output[-1], temperature=0.9)
        generated_text.append(predicted)
        initial_input = predicted.unsqueeze(0)

print('First Citizen:'+decode(torch.stack(generated_text).view(1, -1))[0])
